In [2]:
import os

import numpy as np
import pandas as pd
from z_base import Experiment
from pathlib import Path

In [3]:
%matplotlib qt

In [4]:
results_dir = Path(r'Result_data')
dfs=[]
for file in os.listdir(results_dir):
    exp = Experiment.from_hdf5(results_dir/file)
    dfs.append(exp.d)
DF=pd.concat(dfs).reset_index(drop=True)
DF.head(3)

,compound,T,Nu,rho,w_mass,w_vol,x,V,ln_Nu,1_T,D,ln_D
0,butanol,286.15,3.72,0.82668,100.0,100.0,100.0,4.499927,1.313724,0.003495,76.922043,4.342792
1,butanol,286.35,3.70,0.82668,100.0,100.0,100.0,4.475734,1.308333,0.003492,77.391892,4.348882
2,butanol,286.45,3.70,0.82668,100.0,100.0,100.0,4.475734,1.308333,0.003491,77.418919,4.349231


In [23]:
DF['rho'].unique()[0]-DF['rho'].unique()[1]

-1.1102230246251565e-16

# plots

In [16]:
# Simple data
datas = []
for (compound,x), group in DF.groupby(by=['compound', 'x']):
    for i0 in [15, 20, 25, 30, 40]:
        window_df = group[(273.15 +i0 - 1 <= group['T']) & (group['T'] <= 273.15 +i0 + 1)]
        window_note = window_df.iloc[:,1:].mean()

        window_note['compound'] = compound
        window_note['T'] = 273.15+i0
        if any(window_note.isna()):
            continue
        datas.append(window_note)

DF2 = pd.concat(datas, axis=1).T
# DF2.to_csv('Datas/Viscosity.csv')
DF2

,T,Nu,rho,w_mass,w_vol,x,V,ln_Nu,1_T,D,ln_D,compound
0,288.15,3.601585,0.82668,100.0,100.0,100.0,4.356686,1.281262,0.00347,80.026642,4.382216,butanol
1,293.15,3.19869,0.82668,100.0,100.0,100.0,3.869321,1.162628,0.003411,91.665373,4.518001,butanol
2,298.15,2.82174,0.82668,100.0,100.0,100.0,3.413341,1.037212,0.003354,105.69483,4.66038,butanol
3,303.15,2.509739,0.82668,100.0,100.0,100.0,3.035926,0.920044,0.003299,120.816673,4.794109,butanol
4,313.15,2.036047,0.82668,100.0,100.0,100.0,2.462921,0.710934,0.003194,153.821737,5.035696,butanol
...,...,...,...,...,...,...,...,...,...,...,...,...
138,293.15,64.985317,1.18,19.76,28.205352,49.775459,55.072303,4.173266,0.00341,4.52131,1.507836,peta
139,298.15,48.079533,1.18,19.76,28.205352,49.775459,40.745367,3.872368,0.003354,6.20698,1.825126,peta
140,303.15,36.124262,1.18,19.76,28.205352,49.775459,30.613781,3.586333,0.003299,8.401986,2.127771,peta
141,313.15,21.438908,1.18,19.76,28.205352,49.775459,18.168566,3.064925,0.003196,14.602446,2.680922,peta


In [18]:
res = {}
for compound in DF2["compound"].unique():
    df_comp = DF2[DF2["compound"] == compound]
    xs = df_comp["x"].unique()
    rhos = np.array([df_comp[df_comp["x"] == x]["rho"].mean() for x in xs]) * 1000
    print(f"{compound: >10}:", *xs.astype(int))
    print("*" * 10, *rhos.astype(int))
    res[compound] = list(zip(xs.astype(int), rhos.astype(int)))

   butanol: 100
********** 826
      dmag: 0 11 11 22 22 22 35 35 35 39 39
********** 1070 1070 1070 1070 1070 1070 1070 1070 1070 1070 1070
       ocm: 0 12 12 12 22 22 31 31 31 38 38 44 44 49 49 49 54 54 54 54 58 58 58
********** 1719 1720 1720 1720 1719 1719 1720 1719 1720 1720 1719 1719 1720 1720 1720 1719 1719 1720 1719 1719 1720 1720 1719
      peta: 0 19 19 19 30 30 30 41 49 49 49 49
********** 1180 1180 1180 1180 1180 1180 1180 1180 1180 1180 1180 1180
